# `swift_too` module

## Swift_QueryJob example - Querying and fetching the results of an existing job

### API version = 1.2

#### Author: Jamie A. Kennea (Penn State)

If you wish to retrieve the results of a previously job, this is the task of the QueryJob class. All you need for this class is your username, shared secret and the job number which is reported back when the job is submitted either with `submit()` or `queue()` methods. The jobnumber can be found in the property `jobnumber`.

In [1]:
from swifttools.swift_too import QueryJob, Swift_VisQuery
from time import sleep

I'll define our username and shared secret next because we'll use them twice in this notebook

In [2]:
username = 'myuser'
shared_secret = 'mysharedsecret'

However, if you don't have a user or just don't want to use it, you can submit this request anonymously. In this case you just give the username and shared secret as "anonymous"

In [3]:
username = 'anonymous'
shared_secret = 'anonymous'

First let's submit a simple job so we can get an example jobnumber. In this case I'm just going to do something that isn't too strenuous on the system, let's see if Sgr A* is currently visible, or when it is next visible.

In [4]:
vq = Swift_VisQuery()
vq.username = username
vq.shared_secret = shared_secret
vq.ra,vq.dec = 266.4168, -29.0078
vq.length = 100 # set this long as Sgr A* can go into Sun constraint
vq.hires = False
if vq.queue():
    print(f"Queued job successfully! Jobnumber = {vq.status.jobnumber}")
else:
    print(f"Error: {vq.status.errors}")

Queued job successfully! Jobnumber = 59435


Note that the job has been submitted, but because we used the `queue()` method, it may not have completed yet. If you wait a bit, it will be, but the next step can be run straight away to demonstrate what happens if processing isn't complete.



OK let's see how our job is doing, because the purpose of the QueryJob class is to allow us to fetch a job later, or maybe ASAP. Maybe you don't want to hang around waiting for a bunch of jobs to complete, submit some, have some coffee, come back and fetch, or just want to query the result of an existing job.

In [5]:
qj = QueryJob()
qj.username = username
qj.shared_secret = shared_secret
qj.jobnumber = vq.status.jobnumber
qj.submit()

print(f"Current status of job #{vq.status.jobnumber}: {qj.status}")


Current status of job #59435: Accepted


Note that for the class `QueryJob`, as it does not take many parameters, you can also pass parameters as arguments. If you do this, then the request will be submitted on invokation, so there's no need use the `submit()` method.

In [6]:
while(not qj):
    qj = QueryJob(username,shared_secret,vq.status.jobnumber)
    print(f"Current status of job #{vq.status.jobnumber}: {qj.status}")
    sleep(1)

The `submit` (or argument form) should return `True` if the job has been fetched. It'll return `False` for a number of reasons, firstly if the job is still *Queued* (waiting to be processed) or *Processing*, the status will indicate this. Also the status can be *Rejected*, in the case where you have given bad parameters, or the job no longer exists.


The above "while" will keep polling the job until it is complete. We put in the sleep between requests so as to not over spam the TOO_API server.

OK let's assume that the above now reports "Current status of job #XXX: Accepted", which indicates a successful `QueryJob` call of a completed job. Let's see what our QueryJob class data now looks like.

In [7]:
qj

Parameter,Value
username,anonymous
status,Accepted
jobnumber,59435
errors,[]
warnings,[]
timestamp,2021-11-14 14:53:11
began,2021-11-14 14:53:12
completed,2021-11-14 14:53:13
fetchresult,True
result,Swift_VisQuery object


You can see the data inside the class here presented in table form. The the more useful value here is `result`, which contains the result of the job. In the Python api, this will be a `Swift_VisQuery` object. You can display the result of the job like this:

In [8]:
qj.result

Begin,End,Length
2022-02-03 11:28:00,2022-02-22 14:53:00,"19 days, 3:25:00"


Other parameters that might be of interest are the `timestamp`, `began` and `completed` values, which allow you to find out when the job was submitted, when it started processing and when it finished. For example:

In [9]:
print(f"Job was submitted at {qj.timestamp}, was queued for {(qj.began - qj.timestamp).seconds}s and took {(qj.completed - qj.began).seconds}s to process.")

Job was submitted at 2021-11-14 14:53:11, was queued for 1s and took 1s to process.


As usual, times reported back are UT. That is it! If all is correct, then the following show the upcoming or current *Swift* visibility window for Sgr A*!

In [10]:
_ = [print(win) for win in qj.result]

2022-02-03 11:28:00 - 2022-02-22 14:53:00 (19 days, 3:25:00)


That first date is going to be the first visibility date, which if the target is visible now should be essentially the same as `timestamp` in `QueryJob`, or sometime in the future if the target was constrained when the job was submitted. 

In [11]:
print(qj.result[0][0] - qj.timestamp)

80 days, 20:34:49


Note due to the weird way datetime displays negative timediffs, you might see something like `-1 day, 23:59:19` if the value is negative!